## Import

In [1]:
!pip install kneed
!pip install recmetrics
!pip install csr_utils
!pip install sentence_transformers
import matplotlib.pyplot as plt
import numpy as np
from sentence_transformers import SentenceTransformer
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

import warnings
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from os import listdir
from os.path import isfile, join
import re
import datetime as dt
from itertools import chain
import random
from tqdm import tqdm 
import matplotlib.pyplot as plt
import seaborn as sns
#for collaborative filtering
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import NearestNeighbors
from kneed import KneeLocator
import recmetrics

# # Colab
# from google.colab import drive
# drive.mount('/content/gdrive')

#mypath = '/content/gdrive/MyDrive/public_media_assignment2/data/PBS/'# This is the directory for Tino change make a new line of code to make it work for your own device
#mypath = '/content/gdrive/MyDrive/public_media_assignment2-20230324T124853Z-001/public_media_assignment2/data/PBS/'# thanos
#mypath = '/content/gdrive/MyDrive/Colab Notebooks/public_media_assignment2/data/PBS/'

mypath = 'PBS/' # local
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles = np.sort(onlyfiles)

  Using cached plotly-4.14.3-py2.py3-none-any.whl (13.2 MB)
  Attempting uninstall: plotly
    Found existing installation: plotly 5.14.0
    Uninstalling plotly-5.14.0:
      Successfully uninstalled plotly-5.14.0


## Extracting Data and Preprocessing

In [2]:
objects = []
dfs = []
for i in onlyfiles:
  with (open(mypath + i, "rb")) as openfile:
      while True:
          try:
              dfs.append(pickle.load(openfile))
          except EOFError:
              break
i=0
for df in dfs:
    timestamps = []
    # Extract meta data
    df['season'] = df['meta'].apply(lambda x: np.nan if(re.search(r'(?<=Season\s)(\w+)', x))==None else re.search(r'(?<=Season\s)(\w+)', x)[0]) # What is the season number
    df['episode'] = df['meta'].apply(lambda x: np.nan if(re.search(r'(?<=Episode\s)(\w+)', x))==None else re.search(r'(?<=Episode\s)(\w+)', x)[0]) # What is the episode number
    df['special'] = df['meta'].apply(lambda x: 0 if(re.search(r'Special', x))==None else 1) # If the show/episode is a sepcial or not
    df['clip'] = df['meta'].apply(lambda x: 0 if(re.search(r'Clip', x))==None else 1) # If the show/episode is a clip or not
    df['aired'] = df['meta2'].apply(lambda x: np.nan if(re.search(r'[0-9]{2}\/[0-9]{2}\/[0-9]{2}', x))==None else re.search(r'[0-9]{2}\/[0-9]{2}\/[0-9]{2}', x)[0]) # Match any date format
    df['rating'] = df['meta2'].apply(lambda x: np.nan if(re.search(r'(?<=Rating: ).*$', x))==None else re.search(r'(?<=Rating: ).*$', x)[0]) # Everything after Rating: 

    # Adding the durations of the shows and episodes to the dataframe
    duration = df['meta'].apply(lambda x: np.nan if(re.search(r'[^|]*$', x))==None else re.search(r'[^|]*$', x)[0]) # Select everything after the symbol |
    h = duration.apply(lambda x: np.nan if(re.search(r'(\d+)h', x))==None else re.search(r'(\d+)h', x)[0].strip('h')) # Select everything before the chracter s
    m = duration.apply(lambda x: np.nan if(re.search(r'(\d+)m', x))==None else re.search(r'(\d+)m', x)[0].strip('m')) # Select everything before the chracter m
    s = duration.apply(lambda x: np.nan if(re.search(r'(\d+)s', x))==None else re.search(r'(\d+)s', x)[0].strip('s')) # Select everything before the chracter s
    h = h.fillna(0).astype(int)
    m = m.fillna(0).astype(int)
    s = s.fillna(0).astype(int)
    time_array = np.array(pd.DataFrame({'h':h, 'm':m, 's':s}).values.flatten()).reshape(-1,3)
    for time in time_array:
        if time[0] >23: # check if input of hours is correct
            if time[1]==0: # there was some incorrect input for ours so they are probably mistaken for minutes this is changed here
                time[1]=time[0]
                time[0]=0
        timestamps.append(dt.time(time[0], time[1], time[2]))
    df['duration'] = timestamps

    # replace non data with nan values
    df.replace('No meta data found', np.nan, inplace=True)
    df.replace('Clip', np.nan, inplace=True)
    df.replace('Special', np.nan, inplace=True)
    df.replace('Episode', np.nan, inplace=True)
    df.replace('Season', np.nan, inplace=True)
    df.replace('No show found', np.nan, inplace=True)

    # remove shows that have no information about them first indicator is that there is now show name in the show column so these nan values are removed
    print(onlyfiles[i].strip('.pkl') + ':', df.show.isna().sum(), 'shows not found')
    df = df.loc[~(df.show.isna())]
    # change aired and duration into datetime
    df['aired'] = pd.to_datetime(df['aired'])

    # Only keep shows/episodes that are not clips

    # remove columns without usefull information
    df.drop(columns=['description2', 'meta', 'meta2'], inplace=True)
    objects.append(df)
    i=i+1

# Keep the most recently aired episodes of unique shows
df_arts_music = objects[0].sort_values(['show', 'aired'], ascending=False).drop_duplicates(['show'])
df_culture = objects[1].sort_values(['show', 'aired'], ascending=False).drop_duplicates(['show'])
df_drama = objects[2].sort_values(['show', 'aired'], ascending=False).drop_duplicates(['show'])
df_food = objects[3].sort_values(['show', 'aired'], ascending=False).drop_duplicates(['show'])
df_history = objects[4].sort_values(['show', 'aired'], ascending=False).drop_duplicates(['show'])
df_howto = objects[5].sort_values(['show', 'aired'], ascending=False).drop_duplicates(['show'])
df_indie = objects[6].sort_values(['show', 'aired'], ascending=False).drop_duplicates(['show'])
df_news = objects[7].sort_values(['show', 'aired'], ascending=False).drop_duplicates(['show'])
df_science = objects[8].sort_values(['show', 'aired'], ascending=False).drop_duplicates(['show'])

# # 2 shows with the same name but they are different so the name is changed for arts and music (NPT is local broadcaster)
df_arts_music.show = df_arts_music.show.replace('Shakespeare Uncovered', 'Shakespeare Uncovered NPT') 
df_history.show = df_history.show.replace("The People's Protectors", "The People's Protectors TPT") 
df_history.show = df_history.show.replace("Veterans Coming Home", "Veterans Coming Home History")
df_culture.show = df_culture.show.replace("Veterans Coming Home", "Veterans Coming Home Culture")
df_history.show = df_history.show.replace("Our Town", "Our Town History")
df_howto.show = df_howto.show.replace("Made Here", "Made Here SCETV")
df_news.show = df_news.show.replace("Connecting Point", "Connecting Point NEPM")
df_news.show = df_news.show.replace('Local Focus', 'Local Focus News')
df_news.show = df_news.show.replace('Dialogue', 'Dialogue News')
df_news.show = df_news.show.replace('SnapShot', 'SnapShot KLCS')
df_news.show = df_news.show.replace('State of Change', 'State of Change News')
df_culture.show = df_culture.show.replace('Chasing the Dream', 'Chasing the Dream WSKG')
df_culture.show = df_culture.show.replace('Connect', 'Connect Culture')

# Removing all the shows with unloadable images
# This was an issue with displaying the interface since the whole code wouldn't run anymore
missing_img = 'None?focalcrop=1200x630x50x10&format=auto'
df_arts_music = df_arts_music.loc[~(df_arts_music.image == missing_img)]
df_culture = df_culture.loc[~(df_culture.image == missing_img)]
df_drama = df_drama.loc[~(df_drama.image == missing_img)]
df_food = df_food.loc[~(df_food.image == missing_img)]
df_history = df_history.loc[~(df_history.image == missing_img)]
df_howto = df_howto.loc[~(df_howto.image == missing_img)]
df_indie = df_indie.loc[~(df_indie.image == missing_img)]
df_news = df_news.loc[~(df_news.image == missing_img)]
df_science = df_science.loc[~(df_science.image == missing_img)]

arts-and-music: 586 shows not found
culture: 725 shows not found
drama: 101 shows not found
food: 181 shows not found
history: 630 shows not found
home-howto: 109 shows not found
indie: 87 shows not found
news: 826 shows not found
science: 376 shows not found


In [3]:
df_food.head()

,category,type,show,title,description,image,season,episode,special,clip,aired,rating,duration
2136,Food,episode,tasteMAKERS,Jacobsen Salt Co.,"In Portland, Oregon, Jacobsen Salt Co. harvest...",https://image.pbs.org/video-assets/OfFlP38-ass...,2,13,0,0,2020-04-11,NR,00:26:46
704,Food,episode,Yan Can Cook: Spice Kingdom,Panda-Monium,"In Chengdu, Martin feeds pandas, harvests bamb...",https://image.pbs.org/video-assets/yrqxkUk-ass...,NaN,101,0,0,2021-11-03,TV-G,00:26:46
1130,Food,episode,Wisconsin Foodie,Oneida White Corn,Learn how to prepare White Corn soup and the i...,https://image.pbs.org/video-assets/uHUXXTC-ass...,12,8,0,0,2022-03-24,NR,00:26:47
603,Food,episode,Wine First,Styria & Burgenland,"While exploring Austria, our hosts participate...",https://image.pbs.org/video-assets/0rHRK58-ass...,NaN,103,0,0,2021-10-05,TV-PG,00:27:28
870,Food,episode,Valleys Gold,Dates,Learn about the economic engine that drives th...,https://image.pbs.org/video-assets/x3BLAsZ-ass...,5,12,0,0,2018-12-10,TV-G,00:26:47


## Creating Personas

- Young
    - NO news / food shows / DIY
    - It feels like they are more into entertainment. 
    - They don’t care about what is going on, probably watch the channel more to disconnect.
    - They watch science programs, art/music, drama, history…
- Family people
    - Adults with kids, in their 35-50s.
    - They usually watch the news and a lot of kids' shows (because of their kids).
    - They don’t have that much time to watch more entertainment shows. 
- “Older” people
    - They are mostly retired.
    - They have time to watch more stuff, and more diverse: drama, art/music, DIY, food…
    - Not that much of the news. 
- Lifetime fans
    - They are always on PBS.
    - Generally people in their 50s or around.
    - Conservative.
    - They don’t watch indie films or the news that much. 


In [4]:
# give weights according to what shows the persona likes watching (weight for zero, weight for 1)
# order of genre is df_arts_music, df_culture, df_drama, df_food, df_history, df_howto, df_indie, df_news, df_science
# Weight:0, Weight:1 (not_watched/watched)
young_persona = [
    (70,30), # art and music
    (90,10), # culture
    (60,40), # drama
    (99,1), # food
    (95,5), # history
    (98,2), # how to
    (60,40), # indie
    (95,5), # news
    (90,10) # science
    ]

family_persona = [
    (90,10), # art and music
    (90,10), # culture
    (90,10), # drama
    (80,20), # food
    (85,15), # history
    (80,20), # how to
    (95,5), # indie
    (70,30), # news
    (85,15) # science
    ]

old_persona = [
    (70,30), # art and music
    (90,10), # culture
    (90,10), # drama
    (80,20), # food
    (70,30), # history
    (80,20), # how to
    (95,5), # indie
    (85,15), # news
    (80,20) # science
    ]

fan_persona = [
    (70,30), # art and music
    (85,15), # culture
    (60,40), # drama
    (75,25), # food
    (80,20), # history
    (80,20), # how to
    (95,5), # indie
    (95,5), # news
    (70,30) # science
    ]

## Creating Users

We start by adding ones and zeroes to shows and episodes to indicate if the user watched that show or not. We hold the different personas into account and will keep the genre of the shows in mind when declaring if someone watched a show or not. Also it's important to know that there are shows and episodes. Shows are a 1 time thing and episodes have multiple episodes under the same show name. We can either:

- See every episode as its own show depending on the title
- combine all the episodes and see it as one show

To make it easier we start by combining all the episodes into one show and if the user has watched 1 episode it can be added to 'continue watching' and won't be considered for recommendation

In [5]:
def create_users(dfs, persona, n_users, seed, name): # all dataframes, weights of persona and amount of desired users
    df_users = []
    random.seed(seed)
    for i in range(len(persona)): # Loop through the different genres of the df and weights
        df = pd.DataFrame(columns=dfs[i].show.unique()) # ceate an empty dataframe where the columns represent all the unqiue shows in the current genre
        for n in range(n_users): # For every genre make n_users with according weights
            watched = random.choices([0, 1], weights=persona[i], k=len(df.columns))
            df.loc[name + '_user'+str(n)] = watched
        df_users.append(df) # add the genre of the n_users to a list of dataframes

    df_users = pd.concat(df_users, axis=1, ignore_index=False) # combine all the genres together to create one dataframe for the persona with every possible show
    return df_users

In [6]:
dataframes = [df_arts_music, df_culture, df_drama, df_food, df_history, df_howto, df_indie, df_news, df_science] # all dfs
# Create users
df_young = create_users(dataframes, young_persona, n_users=15, seed=42, name='young')
df_family = create_users(dataframes, family_persona, n_users=20, seed=4, name='family')
df_old = create_users(dataframes, old_persona, n_users=20, seed=69, name='old')
df_fan = create_users(dataframes, fan_persona, n_users=20, seed=666, name='fan')
df_users = pd.concat([df_young, df_old, df_family, df_fan])
df_shows = pd.concat(dataframes).reset_index().drop(columns='index')
df_shows.index.name='movieId'
df_users.head()

,arts IN focus,arts Bridge,Zoltan Maga: Live from Budapest with David Foster,Yoshiki: Live at Carnegie Hall,Yes to Running: Bill Harley Live,Wynonna Judd: Concert from My Place,Writing in Peace and War,Worth Waiting For : Journey of a Musical,Work In Progress,Within/Earshot Jazz Festival,...,Alabama Scholars Bowl,Alabama STEM Explorers,After the Ice,Adaptation,A Year in Space,A Walk in the Park with Nick Mollé: Nature of the Beasts,A Story of Patients,A Force For Nature: Lucy Braun,A Dog's Service,35 Million Years Down the Chickasawhay
young_user0,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
young_user1,0,0,1,1,0,1,0,1,1,0,...,0,1,1,0,0,0,0,0,0,0
young_user2,0,0,1,1,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
young_user3,1,0,1,0,1,1,0,0,0,0,...,1,0,0,1,0,1,0,0,0,0
young_user4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## Creating Watched dataframe

In [7]:
# With this dataframe we can do content based recommendations to start of the recommender system
watched = []
for x in range(len(df_users)):
    for i in range(len(df_users.columns)):
        if df_users.iloc[x,i] == 1:
            watched.append((df_users.index[x], df_users.columns[i]))
df_watched = pd.DataFrame(watched, columns=['user', 'show'])
df_watched.head()

,user,show
0,young_user0,Yes to Running: Bill Harley Live
1,young_user0,Writing in Peace and War
2,young_user0,Welcome Christmas with VocalEssence
3,young_user0,We Remember: Songs of Survivors
4,young_user0,Walter Anderson: The Extraordinary Life and Ar...


## Creating Genre dataframe

In [8]:
df_category = df_shows[['category']]
df_category = pd.crosstab(df_category.index, df_category.category)
df_category.index.name = 'movieId'
print(df_category.shape)
df_category

(3365, 9)


category,Arts & Music,Culture,Drama,Food,History,Home & Howto,Indie,News,Science
movieId,,,,,,,,,
0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
3360,0,0,0,0,0,0,0,0,1
3361,0,0,0,0,0,0,0,0,1
3362,0,0,0,0,0,0,0,0,1


In [9]:
df_users_movies = []
# Go over each user and check if there is a 1 if there is they watched that movie and add it to the users personal watched list
for i in range(len(df_users)):
    user=[]
    for x in range(len(df_users.columns)):
        if df_users.iloc[i,x] == 1:
            user.append(df_category.iloc[x,:])
    # save it as a dataframe and add it to a list
    df = pd.DataFrame(user, columns=df_category.columns)
    df.index.name = df_users.iloc[[i]].index.values[0]
    df.columns.name = 'movieId'
    df_users_movies.append(df)

In [10]:
df_users_movies[10].head()

movieId,Arts & Music,Culture,Drama,Food,History,Home & Howto,Indie,News,Science
young_user10,,,,,,,,,
0,1,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0
10,1,0,0,0,0,0,0,0,0
13,1,0,0,0,0,0,0,0,0
17,1,0,0,0,0,0,0,0,0


## Save dataframes

In [11]:
# Dataframes to csv
#mypath = '/content/gdrive/MyDrive/public_media_assignment2/data' drive
mypath = 'data' #local
# df_young.to_csv(mypath+'/df_young.csv', index=False)
# df_family.to_csv(mypath+'/df_family.csv', index=False)
# df_old.to_csv(mypath+'/df_old.csv', index=False)
# df_fan.to_csv(mypath+'/df_fan.csv', index=False)
df_watched.to_csv(mypath+'/df_watched.csv', index=True)
df_users.to_csv(mypath+'/df_users.csv', index=True)
df_category.to_csv(mypath+'/df_category.csv', index=True)
df_arts_music.to_csv(mypath+'/df_arts_music.csv', index=False)
df_culture.to_csv(mypath+'/df_culture.csv', index=False)
df_drama.to_csv(mypath+'/df_drama.csv', index=False)
df_food.to_csv(mypath+'/df_food.csv', index=False)
df_history.to_csv(mypath+'/df_history.csv', index=False)
df_howto.to_csv(mypath+'/df_howto.csv', index=False)
df_indie.to_csv(mypath+'/df_indie.csv', index=False)
df_news.to_csv(mypath+'/df_news.csv', index=False)
df_science.to_csv(mypath+'/df_science.csv', index=False)
df_shows.to_csv(mypath+'/df_shows.csv', index=False)

We created the dataframe with all the different shows that different users watched, so now a user based dataframe can be created. In PBS there are multiple ways a user can indicate if he/she is interested in a show these are:

- clicked on the show 
- watched the preview
- clicked on explore the show
- added the show to his or her list

Also we can add different user data for example

- at what time the user watched
- the device the user is watching on 
- how long the user watched in a session

With this information a dataframe can be created that indicates what actions different users performed on different shows

In [12]:
# this is pretty advanced should ask in seminar

## Content Based Filtering

Content-based filtering is a technique used in recommender systems that focuses on the attributes of the items being recommended rather than the preferences of the users. It works by analyzing the content or features of the items and then recommending similar items to users who have shown an interest in those features.

The system creates a profile for each user based on the items they have interacted with or rated in the past. The system then uses this information to recommend new items to the user that have similar content or features to the items they have enjoyed in the past. For example, if a user has watched and liked several action movies, the system will recommend more action movies to that user.

We can implement this by recommending shows that the user enjoys for example according to the genre

In [13]:
top_10 = df_watched['show'].value_counts().head(10).reset_index().rename(columns={'index':'show', 'show':'count'})
top_10.to_csv(mypath+'/recommendations-most-watched.csv', index=False)
top_10

,show,count
0,Briarhoppers: Celebrating 86 Years,29
1,The Elders’ Last Tour: Going To Arklow,29
2,American Songbook at NJPAC Hosted by Michael F...,29
3,Landscapes of South Dakota,28
4,The Nitty Gritty Dirt Band: 50 Years and Circl...,28
5,KC Bands Together,27
6,The Arts Page,27
7,Off 90,27
8,Illinois Artists at Work: Cannot Live Without,26
9,Between The Covers,26


## Ianthi

In [14]:
#Combine all categories
dataframes = pd.concat([df_arts_music, df_culture, df_drama, df_food, df_history, df_howto, df_indie, df_news, df_science]) 
#Reseting numbering
dataframes = dataframes.reset_index(drop=True)
#Keeping the categories we are going to use
X = np.array(dataframes.description)
data = dataframes[['category','description','show']]
data.to_csv('data/content_based.csv', index=True)
data.head()

,category,description,show
0,Arts & Music,Featured: Furniture restoration expert Bruce S...,arts IN focus
1,Arts & Music,"Cincinnati Maritime Modelers, artist Amy Deal,...",arts Bridge
2,Arts & Music,Zoltán Mága: Live from Budapest is a musical e...,Zoltan Maga: Live from Budapest with David Foster
3,Arts & Music,Japanese musician Yoshiki packs Carnegie Hall ...,Yoshiki: Live at Carnegie Hall
4,Arts & Music,Bill Harley delights children and adults alike...,Yes to Running: Bill Harley Live


In [15]:
text_data = X
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = model.encode(text_data, show_progress_bar=True)

Batches:   0%|          | 0/106 [00:00<?, ?it/s]

In [16]:
X = np.array(embeddings)
n_comp = 5
pca = PCA(n_components=n_comp)
pca.fit(X)
pca_data = pd.DataFrame(pca.transform(X))
pca_data.head()

,0,1,2,3,4
0,1.819210,1.725434,-0.559228,-4.118472,1.635649
1,-2.827150,-2.580153,-1.213453,-3.791024,-0.493212
2,-2.256783,0.739761,4.654517,-2.813555,-2.306345
3,-6.151664,-3.833063,5.673294,-2.390007,-1.182634
4,1.831611,-5.404703,4.819712,-2.281183,1.834391


In [17]:
#Plotting the PCA
#sns.pairplot(pca_data)

In [18]:
cos_sim_data = pd.DataFrame(cosine_similarity(X))
cos_sim_data.to_csv('data/cos_sim_data.csv', index=True)
def give_recommendations(index,cos_sim_data,number_recom, print_recommendation = False,print_recommendation_description= False,print_category =False):
  recomm = cos_sim_data.loc[index].sort_values(ascending=False).index.tolist()[1:number_recom+1]
  movies_recomm =  data['show'].loc[recomm].values
  result = {'Movies':movies_recomm,'Index':recomm}
  return result

In [19]:
#Saving the 5 recommendations for each show in the list
recomm_list = []
for i in range(len(df_shows)):
  recomm_i = give_recommendations(i,cos_sim_data,10)
  recomm_list.append(recomm_i['Movies'])

# check what shows the user watched
watched_shows = df_watched.loc[df_watched.user=='young_user1'].show.values
movie_ids = df_shows.loc[df_shows['show'].isin(watched_shows)].index

# give recommendations based on watched shows
recomm_data = pd.DataFrame(recomm_list)
recomm_data.to_csv('data/content_based_recom.csv', index=True)
recomm_data['show'] = data['show']
recom_movie = recomm_data.loc[recomm_data.index.isin(movie_ids)]
random_num = random.choice(recom_movie.index)
recom_movie.loc[random_num].values

array(['Lakeland News', 'Eyes on the Prize', 'WGBH', 'Citizen Lane',
       'Seasonal Specials', 'The Newport Bridge: A Rhode Island Icon',
       'SXSW Flashback', 'Global Voices', 'Chicago Tonight: Black Voices',
       'Return of the Cicadas',
       "Everlasting Light: Indiana's Bicentennial Torch Relay"],
      dtype=object)

In [20]:
movie = recom_movie['show'].loc[random_num]
movie

"Everlasting Light: Indiana's Bicentennial Torch Relay"

## Collaborative Filtering

There are two main types of collaborative filtering: 
- user-based:

In user-based collaborative filtering, the system recommends items to a user based on the preferences of similar users. This is done by identifying users who have similar preferences and recommending items that they have enjoyed. 
- item-based:

In item-based collaborative filtering, the system recommends items to a user based on the similarity of the items themselves. This is done by analyzing the characteristics of items and recommending items that are similar to ones the user has enjoyed in the past.

In [21]:
df_users.shape

(75, 3365)

1st step: Create the jaccard similarity matrix for item-based collab filtering

In [22]:
# compute Jaccard similarity score matrix
jaccard_sim = 1 - pairwise_distances(np.array(df_users), metric='jaccard')

jaccard_sim.shape

c:\Users\tinot\anaconda3\envs\rs\lib\site-packages\sklearn\metrics\pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


(75, 75)

In [23]:
#create the dataframe
similarity_df = pd.DataFrame(jaccard_sim, index=df_users.index, columns=df_users.index)

similarity_df.to_csv('data/similarity_df.csv', index=True)

In [24]:
#There were duplicate rows and columns
similarity_df = similarity_df.drop_duplicates()
similarity_df = similarity_df.loc[:,~similarity_df.columns.duplicated()]
#For the user-item matrix as well
df_users.drop_duplicates(inplace= True)

In [25]:
#Seperate personsas dataframes
young_users = similarity_df.loc[similarity_df.index.str.startswith('young_user')]
old_users = similarity_df.loc[similarity_df.index.str.startswith('old_user')]
fam_users = similarity_df.loc[similarity_df.index.str.startswith('family_user')]
fan_users = similarity_df.loc[similarity_df.index.str.startswith('fan_user')]

In [26]:
#save dfs
young_users.to_csv('data/young_users.csv', index=True)
old_users.to_csv('data/old_users.csv', index=True)
fam_users.to_csv('data/fam_users.csv', index=True)
fan_users.to_csv('data/fan_users.csv', index=True)

# **Find the 3 most similar users and movies from other personas!!!**

In [27]:
def find_sim_users(target_user,sim_df,young_users, old_users, fam_users, fan_users):

  # Retrieve the target persona from the target user's index label
  target_persona = target_user.split('_')[0]
  sim_df = sim_df.copy()
  # Define the personas and their corresponding sets of users
  personas = {'young': young_users.index,
            'old': old_users.index,
            'family': fam_users.index,
            'fan': fan_users.index}

  # Retrieve the set of users from the target user's persona
  try:
    users_from_same_persona = personas[target_persona]
  except KeyError:
    raise ValueError(f"Invalid persona '{target_persona}' in target user '{target_user}'")

  # Set the similarity scores of users in the same persona as the target user to 0.0, to consider them as not similar
  sim_df.loc[target_user, sim_df.index.str.startswith(target_persona)] = 0.0

  # Sort the remaining values in the target user's row in descending order
  similar_users_indices = sim_df.loc[target_user, :].sort_values(ascending=False).index 

  # Exclude the target user's index label from the list of similar users indices
  similar_users_indices = similar_users_indices[similar_users_indices != target_user]

  K=3

  top_K_users = {}
  for user_idx in similar_users_indices:
      if len(top_K_users) == K:
            break
      if user_idx not in users_from_same_persona and user_idx not in top_K_users:
            top_K_users[user_idx] = similarity_df.loc[target_user, user_idx]

  return top_K_users

In [31]:
def recommend_movies_user(target_user,similarity_df, user_item_df, young_users, old_users, fam_users, fan_users):
    # get the n nearest similar users for the input user
    nearest_users = find_sim_users(target_user,similarity_df, young_users, old_users, fam_users,fan_users)
    sim_users = list(nearest_users.keys())
    
    # create a list of sets of the movies watched by each similar user
    user_movies = [set(user_item_df.loc[(user, slice(None))].loc[user_item_df.loc[(user, slice(None))] == 1].index) for user in sim_users]

    
    # find the movies watched by any of the similar users
    sim_movies = set.union(*user_movies)
    
    # remove the movies already watched by the input user
    watched_movies = set(user_item_df.loc[target_user].loc[user_item_df.loc[target_user] == 1].index)
    sim_movies = sim_movies - watched_movies
    
    # sort the movies by the number of similar users who watched them and return the top 10
    sorted_sim_movies = sorted([(movie, sum([movie in user_movies[i] for i in range(len(user_movies))])) for movie in sim_movies], key=lambda x: x[1], reverse=True)
    recommended_movies = [movie[0] for movie in sorted_sim_movies[:20] if movie[0] not in watched_movies]

    return recommended_movies


In [32]:
recommend_movies_user("young_user2",similarity_df,df_users,young_users,old_users,fam_users, fan_users)

['Grassland Jam',
 'Front Row Boston',
 'Life at the Waterhole',
 "A Chef's Life",
 'Poetry in America',
 'Painting Taos',
 'Hemingway',
 'The Senator from New Mexico: Pete V. Domenici',
 'Civilization: The West and the Rest with Niall Ferguson',
 'Uranium Twisting the Dragons Tail',
 'Climate Artists',
 'Simply Ming',
 'Arts America',
 'The Cartoon Guy Show - Elementary "Line-scapping"',
 'Great Job!',
 'Symphony for Nature: The Britt Orchestra at Crater Lake',
 'Professor T',
 'Climate Smart: Cities Working Together',
 'Backroad Bites',
 'Breathe Yoga for Kids']

In [34]:
young1 = recommend_movies_user("young_user1",similarity_df,df_users,young_users,old_users,fam_users,fan_users)
fam1 = recommend_movies_user("family_user1",similarity_df,df_users,young_users,old_users,fam_users,fan_users)
old5 = recommend_movies_user("old_user5",similarity_df,df_users,young_users,old_users,fam_users,fan_users)
young5 = recommend_movies_user("young_user5",similarity_df,df_users,young_users,old_users,fam_users,fan_users)
fam6 = recommend_movies_user("family_user6",similarity_df,df_users,young_users,old_users,fam_users,fan_users)

In [35]:
#extract the category and the title of each show
df_category = df_shows[['category',"show"]]

In [36]:
#find the show id of the show in the original dataframe
young1 = df_category.loc[df_category.show.isin(young1)]
fam1 = df_category.loc[df_category.show.isin(fam1)]
old5 = df_category.loc[df_category.show.isin(old5)]
young5 = df_category.loc[df_category.show.isin(young5)]
fam6 = df_category.loc[df_category.show.isin(fam6)]

In [37]:
# Create the dataframe with each show assigned to each category
df_young1 = pd.pivot_table(young1, index="movieId", columns="category", values="show", aggfunc=lambda x: 1, fill_value=0)
df_fam1 = pd.pivot_table(fam1, index="movieId", columns="category", values="show", aggfunc=lambda x: 1, fill_value=0)
df_old5 = pd.pivot_table(old5, index="movieId", columns="category", values="show", aggfunc=lambda x: 1, fill_value=0)
df_young5 = pd.pivot_table(young5, index="movieId", columns="category", values="show", aggfunc=lambda x: 1, fill_value=0)
df_fam6 = pd.pivot_table(fam6, index="movieId", columns="category", values="show", aggfunc=lambda x: 1, fill_value=0)

In [38]:
#concat the two dataframes for each user with the categories features
result_df = pd.concat([df_young1, df_fam1, df_old5,df_young5,df_fam6])

In [39]:
result_df.fillna(0,inplace= True)
result_df = result_df.astype(int)

In [40]:
list1 = df_young1.index.tolist()
list2 = df_fam1.index.tolist() 
list3 = df_old5.index.tolist() 
list4 = df_young5.index.tolist() 
list5 = df_fam6.index.tolist() 
combine = [list1,list2,list3,list4,list5]
#remove any empty lists
combine = [lst for lst in combine if lst]

In [41]:
#We should do it with jaccard similarity
from sklearn.metrics import jaccard_score

# Define the function to compute the Jaccard similarity between two lists
def jaccard_similarity(x, y):
    return jaccard_score(set(x), set(y))
# Compute the intra-list similarity using Jaccard similarity for each dataframe
#similarity_scores_list = []
#for df in recommendations_list:
similarity_scores = recmetrics.intra_list_similarity(combine,result_df)
#similarity_scores_list.append(similarity_scores)

**SIMPLE COLLAB FILTERING: WITHOUT CLUSTERS**

In [42]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(similarity_df.values)
distances, indices = knn.kneighbors(similarity_df.values, n_neighbors=4)

In [43]:
def find_similar_users(user_name, similarity_matrix, indices, distances):
    # get the index for the input user
    index_for_user = similarity_matrix.index.tolist().index(user_name)
    
    # find the indices for the similar users
    sim_user_indices = indices[index_for_user].tolist()
    
    # distances between the input user and the similar users
    user_distances = distances[index_for_user].tolist()
    
    # the position of the input user in the list sim_users
    id_user = sim_user_indices.index(index_for_user)
    
    # remove the input user from the list sim_users
    sim_user_indices.remove(index_for_user)
    
    # remove the input movie from the list movie_users
    user_distances.pop(id_user)
    
    # get the user names for the similar users
    sim_user_names = similarity_matrix.iloc[sim_user_indices].index.tolist()
    
    # create a dictionary with the nearest users and their distances
    nearest_users = {sim_user_names[i]: user_distances[i] for i in range(len(sim_user_names))}
    
    return nearest_users


In [44]:
find_similar_users("young_user12", similarity_df, indices, distances)

{'young_user1': 0.45079193536322526,
 'young_user13': 0.4508171093558916,
 'young_user3': 0.45229225901327763}

In [45]:
def recommend_movies(user_name, similarity_matrix, indices, distances, user_movie_df, n=3):
    # get the n nearest similar users for the input user
    nearest_users = find_similar_users(user_name, similarity_matrix, indices, distances)
    sim_users = list(nearest_users.keys())[:n]
    
    # create a list of sets of the movies watched by each similar user
    user_movies = [set(user_movie_df.loc[user].loc[user_movie_df.loc[user] == 1].index) for user in sim_users]
    
    # find the movies watched by any of the similar users
    sim_movies = set.union(*user_movies)
    
    # remove the movies already watched by the input user
    watched_movies = set(user_movie_df.loc[user_name].loc[user_movie_df.loc[user_name] == 1].index)
    sim_movies = sim_movies - watched_movies
    
    # sort the movies by the number of similar users who watched them and return the top 10
    sorted_sim_movies = sorted([(movie, sum([movie in user_movies[i] for i in range(len(user_movies))])) for movie in sim_movies], key=lambda x: x[1], reverse=True)
    recommended_movies = [movie[0] for movie in sorted_sim_movies[:20] if movie[0] not in watched_movies]

    return recommended_movies


In [46]:
#Create a function to find the most similar shows("neighbours") of a specific show.
#Take as input a movie name, the similarity matrix, 
#the indices and the distances as computed in the knn 

def similar_movies(movie_name, sim_matrix, indices, distances, num_neighbors):
    # get the index for the input movie
    index_for_movie = sim_matrix.index.tolist().index(movie_name)
    
    # find the indices for the similar movies
    sim_movie_indices = indices[index_for_movie][1:num_neighbors+1] # exclude the first index (which is the movie itself)
    
    # get the names of the similar movies from the similarity matrix
    sim_movie_names = [sim_matrix.index[i] for i in sim_movie_indices]
    
    # get the distances between the input movie and the similar movies
    sim_movie_distances = distances[index_for_movie][1:num_neighbors+1] # exclude the first distance (which is the distance to itself)
    
    # create a dictionary with the nearest movies and their distances
    nearest_movies = {sim_movie_names[i]: sim_movie_distances[i] for i in range(len(sim_movie_names))}
    
    return sim_movie_names, nearest_movies


In [47]:
#Apply the algorithm to a movie you want:
k=3
knn = NearestNeighbors(metric='jaccard')
knn.fit(similarity_df)
distances, indices = knn.kneighbors(similarity_df, n_neighbors=k)
similar_movies("Wild Arizona", similarity_df, indices, distances, k)

c:\Users\tinot\anaconda3\envs\rs\lib\site-packages\sklearn\metrics\pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


ValueError: 'Wild Arizona' is not in list

2nd step: Generate recommendations based on user's preferences from the user-item matrix df_users and suggest him/her shows that has never watched before based on the similarity with the shows he/she has watched before.

In [48]:
#Function that gets the watched shows from a user, then finds the 3 nearest neighbors of each watched show.
#After creates a dictionary of similar show based on these neighbors and sort the dictionary by distance.
#Returns a list of 10 most similar shows that the user has not yet watched.
def recommendations(user, similarity_df, indices, distances):
    watched_movies = df_users.loc[user, df_users.loc[user,:] == 1].index.tolist()
    sim_movies = {}
    n_neighbors = 3
    for movie in watched_movies:
        sim_movie_names, nearest_movies = similar_movies(movie, similarity_df, indices, distances, n_neighbors)
        for sim_movie, distance in zip(sim_movie_names, nearest_movies):
            if sim_movie not in watched_movies and sim_movie not in sim_movies:
                sim_movies[sim_movie] = distance
    sorted_sim_movies = sorted(sim_movies.items(), key=lambda x: x[1])
    recommended_movies = [movie[0] for movie in sorted_sim_movies[:20]]

    return recommended_movies

In [49]:
young_user6 = recommendations("young_user6", similarity_df, indices, distances)
print(young_user6)

ValueError: "What's Brewing with Cliff" is not in list

In [50]:
#Create a function that create a list of recommendations for each user by keeping the show id
def list_recommendations(user, df_original):
  user_list = recommendations(user,similarity_df, indices,distances)
  user_data = df_original.loc[df_original.show.isin(user_list)]
  features_df = user_data.pivot_table(index="movieId", columns='category', values='show', aggfunc=len, fill_value=0)
  features_df.columns.name = None
  index_list = features_df.index.tolist()
       
  return index_list

In [51]:
list_recommendations("young_user1", df_shows[["category","show"]])

ValueError: 'Zoltan Maga: Live from Budapest with David Foster' is not in list

## Questions

- These points below are data we can add. I was thinking of creating multiple sparse matrices that indicates with ones and zeroes if they performed the tasks listen below. Based on these matrices we can create recommendations. When there is a good baseline we collect this data by collecting the interactions of the users.
  clicked on the show 
  - watched the preview
  - clicked on explore the show
  - added the show to his or her list

- Also we can add different user data for example

  - at what time the user watched
  - the device the user is watching on 
  - how long the user watched in a session

- Questions
  - show our personas and how we created our dataframe
  - should the interface only show the recommendations?
    - or also the description after clicking on a series?
    - 

  - Value: exposure diversity
    - exposure the amount they can watch (accesible interface)
    - content diversity: watch different things (different categories)
    - Do recommendations that are similar from the same and different categories

In [52]:
df_users = pd.read_csv('data/df_users.csv', encoding='latin-1', index_col=0)
all_users = df_users.index.values